# Analyse de sentiments de commentaires de films par Machine Learning :
***

- ## Objectif de l'étude : 
Faire le tri entre avis positifs et avis négatifs des commentaires de la base de données [IMDB](https://www.imdb.com/) (Internet Movie DataBase) qui contient, parmi bien d'autre choses, des avis sur un grand nombre de films. Puis être capable de prédire automatiquement si un avis est positif ou négatif.

- ## Méthodologie :
Nous allons donc effectuer une classification binaire (1, positif ou 0, négatif) par Natural Language Processing (NLP). Nous choisissons pour cette tâche d'utiliser un réseau de neurones très simple, construit avec le module Keras (API la plus populaire pour le Deep Learning, basée sur TensorFlow). Par chance (ou pas !), le dataset IMDB est disponible dans Keras, pratique ! Il contient les avis de 25000 films. Il s'agit donc ici de Machine Learning supervisé, car le réseau de neurones va apprendre à partir des avis (associés aux commentaires) déjà existants dans le dataset, pour ensuite prédire par lui-même !
***


Commençons par importer les modules Python nécessaires : *NumPy* (module de calcul scientifique), et quelques modules de *Keras* (module de Deep Learning inclus dans *TensorFlow*) tels que ``imdb``, et c'est tout ! <br>
__NB__ : il faudra donc avoir installé les modules *NumPy* et *TensorFlow* au préalable.

In [2]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence

seed = 12345
np.random.seed(seed)



## Le dataset IMDB

### Chargement des données
On télécharge les 1000 premiers mots du dataset, et on découpe ce dataset en deux : un dataset d'entraînement, et un dataset de test, qui va nous permettre d'évaluer la précision de notre réseau de neurones. <br> ``X`` étant nos commentaires, et ``y`` les avis.

In [3]:
voc_size = 1000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = voc_size)

Que contient concrètement ``X`` ?
Regardons

In [4]:
X_train

array([list([1, 14, 22, 16, 43, 530, 973, 2, 2, 65, 458, 2, 66, 2, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 2, 2, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2, 19, 14, 22, 4, 2, 2, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 2, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2, 2, 16, 480, 66, 2, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 2, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 2, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 2, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 2, 88, 12, 16, 283, 5, 16, 2, 113, 103, 32, 15, 16, 2, 19, 178, 32]),
       list([1, 194, 2, 194, 2, 78, 228, 5, 6, 2, 2, 2, 134, 26, 4, 715, 8, 118, 2, 14, 394, 20, 13, 119, 954, 189,

On peut être surprit au premier regard, il n'y a pas de texte !


**Explication :**
Keras nous a prémâché le travail en encodant chaque mot du data set selon sa fréquence d'utilisation (processus appelé *Tokenization*).

Par exemple, "2", "33" qui sont les 1er mots extraits du data set signifient :
2 et 33ème mot les plus fréquents. Et ainsi de suite.

Maintenant que contient ``y`` (les avis) ?

In [5]:
y_train

array([1, 0, 0, ..., 0, 1, 0], dtype=int64)

Ce sont les avis : <br>
1 : positif
0 : négatif

C'est ce qu'on veut faire apprendre à notre réseau de neurones

## Préparer le data set
Une opération classique en Text Mining avec Keras est de *pader* les séquences (morceaux de phrases considérés).

Les séquences trop courtes sont complétées avec des zéros (par défaut). Celles qui sont trop longues sont tronquées.
C'est un moyen d'obtenir un tableau 2D de longueur fixe (1 avis, 1 ligne).

Les paramètres :

In [6]:
voc_size = 1000
input_length = 100
output_dim = 32

In [7]:
X_train = sequence.pad_sequences(X_train, maxlen = input_length)
X_test = sequence.pad_sequences(X_test, maxlen = input_length)

## Créer un modèle de Deep Learning avec Keras

La couche **embedding** permet de transformer chaque mot en vecteur (colonne de chiffres) en tenant compte de son contexte dans la séquence, elle a pour paramètres :
* un vocabulaire de longueur ``voc_size`` (nombre de mots pris en compte)
* des séquences de longueur  ``input_length`` (longueur des morceaux de phrase considérés)
* un vecteur de dimension ``output_dim`` qui contient le mot et son contexte


In [8]:
# create the model
model = Sequential()
model.add(Embedding(voc_size, output_dim, input_length = input_length))
model.add(Flatten())



### Le réseau de neurones
On construit maintenant notre réseau de neurones en ajoutant des couches de neurones (``Dense``), puis on le compile avec la fonction ``compile``.

In [9]:
model.add(Dense(10, activation= 'relu' ))
model.add(Dense(1, activation= 'sigmoid' ))
model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])



### Visualiser le modèle 
On visualise le schéma de notre réseau de neurones

In [10]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 32)           32000     
_________________________________________________________________
flatten (Flatten)            (None, 3200)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                32010     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 64,021
Trainable params: 64,021
Non-trainable params: 0
_________________________________________________________________
None


Ce réseau de neurones très simple contient déjà plus de 64000 paramètres !

## Entrainer l'algorithme sur les données
On entraîne notre modèle sur les données d'entraînement (*train*), et on utilise les données de *test* pour tester/valider sa précision sur des données exogènes (que le modèle n'a jamais vues)

In [11]:
history = model.fit(X_train, 
                    y_train, 
                    validation_data=(X_test, y_test), 
                    epochs = 10, 
                    batch_size = 64,
                    verbose = 1)



Epoch 1/10
391/391 [==============================] - 2s 6ms/step - loss: 0.5034 - accuracy: 0.7386 - val_loss: 0.3825 - val_accuracy: 0.8239
Epoch 2/10
391/391 [==============================] - 2s 5ms/step - loss: 0.3402 - accuracy: 0.8513 - val_loss: 0.3853 - val_accuracy: 0.8222
Epoch 3/10
391/391 [==============================] - 2s 5ms/step - loss: 0.2720 - accuracy: 0.8893 - val_loss: 0.4191 - val_accuracy: 0.8117
Epoch 4/10
391/391 [==============================] - 2s 5ms/step - loss: 0.1910 - accuracy: 0.9311 - val_loss: 0.4854 - val_accuracy: 0.8016
Epoch 5/10
391/391 [==============================] - 2s 5ms/step - loss: 0.1135 - accuracy: 0.9685 - val_loss: 0.5739 - val_accuracy: 0.7939
Epoch 6/10
391/391 [==============================] - 2s 5ms/step - loss: 0.0574 - accuracy: 0.9895 - val_loss: 0.6871 - val_accuracy: 0.7872
Epoch 7/10
391/391 [==============================] - 2s 5ms/step - loss: 0.0261 - accuracy: 0.9974 - val_loss: 0.7869 - val_accuracy: 0.7871
Epoch 

## Evaluer la performance de l'algorithme

On calcule la précision du modèle après entraînement

In [12]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Acc =  %.2f%%" % (scores[1]*100))

Acc =  78.70%


Et voilà ! On voit qu'en seulement quelques lignes de code on a réussi à classifier les avis des commentaires avec une précision de presque 80% !